In [1]:
%run stdPackages.ipynb

# Social Security Design - Argentina

## 1. Input data and simple calibrations

We assume that we have data on the following:
* Worker-to-retiree ratios $\nu_t$.
* Capital income share $\alpha$.
* Frisch elasticity of labor $\xi$ (individual response to after-tax wages).
* Share of retirees that only receive basic pension benefits - here defined as share of hand-to-mouth households ($\gamma_u$).
* Heterogeneity in working households:
    1. Relative sizes of household types $\gamma_i$, such that $\sum_i = \gamma_i$.
    2. Income distribution across types $z_i^{\eta}$.
    3. Distribution of working hours across types $z_i^{x}$.
* The economy wide average savings rate $\overline{s}_t$.
* The ratio of income from $u$-types to the average worker for young ($\chi_1$) and old ($\chi_2$).
* The pension tax in a given year $\overline{\tau}$.
* The target for ratios of replacement rates for 1st and 2nd quartiles $RR$.

### 1.1. Fixed data

In [2]:
dates_log = pd.Index([1950, 1980, 2010, 2040, 2070, 2100], name = 't')
ν_log = np.array([1.497025777, 1.365562914, 1.20756547, 1.110843373, 1.072547602, 1.0725])
T_log = len(ν_log) # number of years in data
T = T_log+3
dates = dates_log.union(pd.Index([dates_log[-1]+30*i for i in range(1,T-T_log+1)]))
ν = np.hstack([ν_log, np.full(T-T_log, ν_log[-1])])
A = np.ones(T) # normalize exog. productivity at 1 
α = .43 # capital income share
ξ = .35 # frisch elasticity
γ = np.full((4, ), 0.25) # four equally large shares
ni = len(γ) # number of types of working households in this case
hours = np.array([38.6, 41.8, 43.4, 46.8]) # weekly hours, working households
income= np.array([618.6, 945.5, 1278.6, 2341.6]) # income levels, working households
zx = hours/hours.mean() # normalized data
zη = income/income.mean() # normalized data
γu =.32 # share of u households
χ1 = 0.3089 # relative income of u-types
χ2 = 0.95 * χ1 # relative income of u-types when old - assumed slightly lower
τ0 = 0.142 # target level of pension tax
s0 = 0.184 # target savings rate
t0date = 2010
t0 = dates.get_loc(t0date) # index for year of calibration 
RR = 0.678/0.803 # replacement rate
ω = 4
ρ = .5
ωu = .3
ωη = 1.65

### 1.2. Calibration of $\eta_i, X_i, \theta$

1. The vector of $(\eta_i/X_i)^{\xi}$ is proportional to eigenvector $\textbf{y}^x$ - i.e. set $(\eta_i/X_i)^{\xi} = y_i^x$.
2. The vector of $(\eta_i^{1+\xi}/X_i^{\xi})$ is proportional to the eigenvector $\textbf{y}^{\eta}$ - i.e. set $\eta_i^{1+\xi}/X_i^{\xi} = k y_i^{\eta} $, where $k>0$ is some constant.
3. We then have $\eta_i =k y_i^{\eta} / y_i^x$ and $(\eta_i/X_i)^{\xi} = y_i^x$. Use $k$ to normalize such that 
$$\begin{align}
    \sum_i \gamma_i \eta_i^{1+\xi}/X_i^{\xi} = 1 \qquad \Rightarrow \qquad k = \dfrac{1}{\sum_i \gamma_i y_i^{\eta}}.
\end{align}$$

Find eigenvectors:

In [3]:
valx, vecx = scipy.sparse.linalg.eigs(zx.reshape(ni,1) * γ.reshape(1,ni), k=1)
valη, vecη = scipy.sparse.linalg.eigs(zη.reshape(ni,1) * γ.reshape(1,ni), k=1)
yx, yη = abs(np.real(vecx)).reshape(ni), abs(np.real(vecη)).reshape(ni) # this assumes that all coordinates in eigenvector are either positive or negative; this should be the case

Calibrate parameters:

In [4]:
η = yη/(yx*sum(γ*yη))
X = η/yx**(1/ξ)

Now given parameters that define household heterogeneity, we define $\theta$ from the relative replacement rates:

In [5]:
Q1 = η[0]**(1+ξ)/(X[0]**ξ)
Q2 = η[1]**(1+ξ)/(X[1]**ξ)
θ = (RR/Q1-1/Q2)/(1-RR+RR/Q1-1/Q2)

### 1.3. Initial guesses for yet-to-calibrated parameters

Set uniform $\beta$ for all types:

In [6]:
β = np.full(ni, fill_value = 1)
βu= min(β) # set impatience equal to lowest producticity household. 

Given our guess on $\beta$, we can define $\epsilon$ from the built-in function for the Argentina case: 

In [7]:
eps = CRRA.argentinaCalEps(θ, β[0])

## 2. Calibration of PEE model

Grid settings:

In [8]:
ngrid = 50 # number of gridpoints in the savings grid
_min = 1e-4 # use as "small" number instead of 0
exp = 1 # nonlinearity in grid, exp>1 increases number of gridpoints in the lower end of the grid

Initialize CRRA and log models:

In [9]:
kwargs = {'α': α, 'A': A, 'ν': ν, 'η': η, 'γ': γ, 'X': X, 'β': β, 'βu': βu, 'ξ': ξ, 'eps': eps, 'θ': θ, 'γu': γu, 'χ1': χ1, 'χ2': χ2, 'ρ': ρ, 'ω': ω, 'ωu': ωu,'ωη': ωη}
m = CRRA.infHorizon(ni=ni, T = T, ngrid = ngrid, **kwargs)
mLog = logModelESC.infHorizon(ni = ni, T = T, **kwargs)
sGrid = CRRA.polGrid(_min, m.solve_ss(0,0,0,ν[-1])['s'], m.ngrid, exp)

Calibration of pure PEE models in the two instances:

In [10]:
m.argentinaCal_simple_PEE(τ0, s0, t0, sGrid);
mLog.argentinaCalibrate(τ0, s0, t0);

Solve baseline:

In [11]:
m.db.update(m.solve_PEE(sGrid))
m.reportAll()
mLog.db.update(mLog.solve_PEE())
mLog.reportAll()

Store solutions for later:

In [12]:
fulldb = m.db.copy()
fulldbLog = mLog.db.copy()

## 3. Unexpected changes to $\theta, \epsilon$ in PEE model

The unexpected changes arrive in 2010. The easiest way to introduce this is to initialize a new model that runs from 2010-2100:

In [13]:
dates_2010 = dates[dates>=2010]
T_2010 = len(dates_2010)
mLog_2010 = logModelESC.infHorizon(ni = ni, T = T_2010, 
                                    eps = fulldbLog['eps'].loc[t0:].set_axis(mLog.db['t'][t0:]-t0), 
                                    θ = mLog.db['θ'].loc[t0:].set_axis(mLog.db['t'][t0:]-t0),
                                    **({k: fulldbLog[k] for k in mLog.defaultParameters} | {'ν': ν[(T-T_2010):],
                                                                                            'A': A[(T-T_2010):]}))
m_2010 = CRRA.infHorizon(ni = ni, T = T_2010, ngrid = ngrid, 
                         eps = fulldb['eps'].loc[t0:].set_axis(m.db['t'][t0:]-t0),
                         θ   = fulldb['θ'].loc[t0:].set_axis(m.db['t'][t0:]-t0),
                         **({k: fulldb[k] for k in m.defaultParameters} | {'ν': ν[(T-T_2010):],
                                                                           'A': A[(T-T_2010):]}))

For the baseline CRRA model, we define the following small function that automatically stores the solution to be used as initial values in the next iteration:

In [14]:
def solveAndUpdatex0_PEE(m, s, s0 = None, x0_t = False, tkwargs = None):
    sol, pol = m.solve_PEE(s, s0 = s0, returnPols = True, x0_t = x0_t, tkwargs = tkwargs)
    m.db.update(sol)
    m.reportAll()
    m.x0['steadyState_PEE'] = pol[m.T-1]['τ'] # steady state search starts from solution
    m.x0['PEEvec_t'] = {t: pol[t]['τ'] for t in m.db['t']} # add dictionary of solutions for each t.
    return m

Solve and store the baseline solution with the shifted time index:

In [15]:
m_2010 = solveAndUpdatex0_PEE(m_2010, sGrid, s0 = fulldb['s[t-1]'][t0]) # solve, report, update initial values
mLog_2010.db.update(mLog_2010.solve_PEE())
mLog_2010.reportAll(s_ = fulldbLog['s'][t0]) # note that the 's' variable in the log model is defined as s[t-1].

Save baseline solutions for later:

In [16]:
base = m_2010.db.copy()
baseLog = mLog_2010.db.copy()

### 3.1. Aux. methods

Provide some labels to call the $i$ types of agents when plotting:

In [17]:
typeLabels = {0: '1st Quartile', 1: '2nd Quartile', 2: '3rd Quartile', 3: '4th Quartile'}

Define a few auxiliary functions used for plotting:

In [18]:
def addYears(x, copy = True):
    """ Replace integer set 't' (from 0 to T) with years in 'dates' """
    if copy:
        x = x.copy()
    if isinstance(x, pd.Index):
        return x.map(pd.Series(dates.values, index = x)).rename('')
    elif isinstance(x, (pd.DataFrame, pd.Series)):
        x.index = addYears(x.index, copy = False)
        return x[dates_log] 
def adjLabels(x, copy = True):
    """ Replace integer set 'i' (from 0 to ni) with labels in 'typeLabels' """
    if copy:
        x = x.copy()
    if isinstance(x, pd.Index):
        return x.map(typeLabels)
    elif isinstance(x, pd.Series):
        x.index = adjLabels(x.index, copy = False)
    elif isinstance(x, pd.DataFrame):
        x.columns = adjLabels(x.columns, copy = False)
    return x
def extractDf(sol, x, grid, d):
    """ Collect data on variables from different simulations in one dataframe """
    return addYears(pd.concat([soli[x] for soli in sol.values()], axis=1), d).set_axis(grid, axis = "columns")    

Whenever we do one of these shocks to a parameter, e.g. $\epsilon$, we want to do some of the same things:
1. Update parameter values in active model. 
2. Solve and store PEE solution.
3. Solve and store equivalent variations.
4. Reset taxes to some baseline value and solve for counterfactual equilibrium (what is the effect of parameter changes *given* fixed tax rates). Store this as well.

In [19]:
#NB: We have turned the updates in initial values for EV, because we do a loop over 2d grid later where large jumps in parameter values occur.
def parameterShock_PEE(m, s, s0, base, parDict):
    m.db.update(parDict) # update parameters
    m = solveAndUpdatex0_PEE(m, s, s0 = s0, x0_t = True, tkwargs = {'ite_PEEvec_t': False}) # solve with initial guess from last iteration
    EVnp = m.solve_EV_Permanent(base, m.db, x0 = m.x0['EV']) # Solve for EV and return as stacked numpy vector 
    # m.x0['EV'] = EVnp # update initial guess for next iteration
    EV = m.ns['EV'].unloadSol(EVnp) # create dictionary with pandas solutions
    sol = (m.db | m.EV_solInPercentages(base, EV)).copy() # merge data to one database
    
    # counterfactual eq. given baseline taxes:
    [m.db.__setitem__(k, base[k].copy()) for k in ('τ', 'τ[t+1]')]; # reset taxes to baseline
    m.db.update(m.solve_EE(m.db['τ'].values, 
                           m.db['θ'].values, 
                           m.db['eps'].values, s0, x0_EE = np.hstack([m.db[k].values for k in m.ns['EE'].symbols]))) # Identify economic equilibrium and add to main database.
    m.reportAll() # report auxiliary variables as well. 
    sol_cf = m.db.copy()
    EVnp = m.solve_EV_Permanent(base, m.db, x0 = m.x0['EVcf'] if 'EVcf' in m.x0 else np.zeros(m.ns['EV'].len)) # Solve for EV and return as stacked numpy vector
    # m.x0['EVcf'] = EVnp
    EV = m.ns['EV'].unloadSol(EVnp)
    sol_cf.update(m.EV_solInPercentages(base, EV))
    return sol, sol_cf

We define a similar method for the log case (this is a bit simpler because we do not need to update initial values along the way):

In [23]:
def log_parameterShock_PEE(m, s0, base, parDict):
    m.updateSolve_PEE(**parDict)
    m.reportAll(s_ = s0)
    EV = m.ns['EV'].unloadSol(m.solve_EV_Permanent(base, m.db))
    sol = (m.db | m.EV_solInPercentages(base, EV)).copy()
    # Reset tax rates to compute counterfactual solution:
    [m.db.__setitem__(k, base[k].copy()) for k in ('τ', 'τ[t+1]')];
    m.db.update(m.solve_EE(m.db['τ'], m.db['τ[t+1]'], m.db['eps[t+1]'], m.db['θ[t+1]']))
    m.reportAll(s_ = base['s'].iloc[0])
    sol_cf = m.db.copy()
    # Compute EV for this scenario as well
    EV = m.ns['EV'].unloadSol(m.solve_EV_Permanent(base, m.db))
    sol_cf.update(m.EV_solInPercentages(base, EV))
    return sol, sol_cf

### 3.2. Simulate effects of gradually changing $\epsilon$ and $\theta$.

Consider varying $\epsilon$ between $0-\epsilon_{max}$ for $\theta\in[0, 1]$. We define: $\epsilon_{max} \equiv \theta \underline{\eta}\underline{h}_{t-1}/h_{t-1}+1-\theta$.

In [21]:
ϵMax   = fulldb['θ'].xs(t0) * min(η) * fulldb['hi'].xs(t0-1).xs(0) / fulldb['h'].xs(t0-1) + 1-fulldb['θ'].xs(t0)
ϵgrid  = pd.Index([round(x,2) for x in np.linspace(0,ϵMax,6)], name = '$\eps$').insert(0, 1-base['θ'].xs(0)).sort_values()
ϵgrid_ = ϵgrid.insert(0, base['eps'].xs(0)) 
θgrid  = pd.Index([round(x,2) for x in np.linspace(0,1,6)], name = '$\\theta$')
θgrid_ = θgrid.insert(0, base['θ'].xs(0))
idx  = pd.MultiIndex.from_product([θgrid, ϵgrid])
idx_ = pd.MultiIndex.from_product([θgrid_, ϵgrid_])
sol = dict.fromkeys(idx_)
solLog = dict.fromkeys(idx_)
sol_cf = dict.fromkeys(idx_)
sol_cfLog = dict.fromkeys(idx_)

Start by identifying a baseline solution with the baseline value of $\epsilon$ and different $\theta$ values. Now, loop over $\epsilon$ values and solve and compare to the relevant baseline. 

In [22]:
%%time
for θ, ϵ in sol:
    sol[(θ, ϵ)], sol_cf[(θ,ϵ)] = parameterShock_PEE(m_2010, sGrid, fulldb['s[t-1]'][t0], base, m_2010.initSC(ϵ, 'eps') | m_2010.initSC(θ, 'θ'))

CPU times: total: 2min 35s
Wall time: 2min 36s


Do the same for the log-case:

In [24]:
%%time
for θ, ϵ in sol:
    solLog[(θ, ϵ)], sol_cfLog[(θ,ϵ)] = log_parameterShock_PEE(mLog_2010, fulldbLog['s'][t0], baseLog, mLog_2010.initSC(ϵ, 'eps') | mLog_2010.initSC(θ, 'θ'))

CPU times: total: 1min 55s
Wall time: 1min 56s
